In [9]:
import pandas as pd

df = pd.read_csv("marathon_results_2017.csv", index_col="Overall")

In [10]:
df.head()

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,Half,twentyfive,thirty,thirtyfive,forty,Pace,Proj Time,Official Time,Gender,Division
Overall,,,,,,,,,,,,,,,,,,,,,
1,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,1:04:35,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,-,2:09:37,1,1
2,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,1:04:35,1:16:59,1:33:01,1:48:19,2:03:14,0:04:58,-,2:09:58,2,2
3,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,1:04:36,1:17:00,1:33:01,1:48:31,2:03:38,0:04:59,-,2:10:28,3,3
4,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,1:04:45,1:17:00,1:33:01,1:48:58,2:04:35,0:05:03,-,2:12:08,4,4
5,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,1:04:35,1:16:59,1:33:01,1:48:41,2:05:00,0:05:04,-,2:12:35,5,5


In [8]:
df.columns

Index(['Bib', 'Name', 'Age', 'M/F', 'City', 'State', 'Country', 'Citizen',
       'Unnamed: 9', 'five', 'ten', 'fifteen', 'twenty', 'Half', 'twentyfive',
       'thirty', 'thirtyfive', 'forty', 'Pace', 'Proj Time', 'Official Time',
       'Overall', 'Gender', 'Division'],
      dtype='object')

In [11]:
splits = df[['Name', 'five', 'ten', 'fifteen', 'twenty', 'Half',
       'twentyfive', 'thirty', 'thirtyfive', 'forty', 'Pace', 'Official Time']]

In [12]:
splits.head()

,Name,five,ten,fifteen,twenty,Half,twentyfive,thirty,thirtyfive,forty,Pace,Official Time
Overall,,,,,,,,,,,,
1,"Kirui, Geoffrey",0:15:25,0:30:28,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:02:53,0:04:57,2:09:37
2,"Rupp, Galen",0:15:24,0:30:27,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:19,2:03:14,0:04:58,2:09:58
3,"Osako, Suguru",0:15:25,0:30:29,0:45:44,1:01:16,1:04:36,1:17:00,1:33:01,1:48:31,2:03:38,0:04:59,2:10:28
4,"Biwott, Shadrack",0:15:25,0:30:29,0:45:44,1:01:19,1:04:45,1:17:00,1:33:01,1:48:58,2:04:35,0:05:03,2:12:08
5,"Chebet, Wilson",0:15:25,0:30:28,0:45:44,1:01:15,1:04:35,1:16:59,1:33:01,1:48:41,2:05:00,0:05:04,2:12:35


In [38]:
def to_seconds(s):
    times = s.split(":")
    return int(times[0])* 3600 + int(times[1])* 60 + int(times[2])

In [39]:
split_cols = ['five', 'ten', 'fifteen', 'twenty',
       'Half', 'twentyfive', 'thirty', 'thirtyfive', 'forty']

df["five_s"] = df.five.map(to_seconds)

ValueError: invalid literal for int() with base 10: '-'

Wait, there are `"-"`'s in there?

In [89]:
# splits = df[split_cols]

for col in split_cols:
    df = df[~df[col].str.contains("-")]
    
df.head()

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,thirtyfive_s,forty_s,ten_split,fifteen_split,twenty_split,twentyfive_split,thirty_split,thirtyfive_split,forty_split,five_split
Overall,,,,,,,,,,,,,,,,,,,,,
1,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,6499,7373,903,916,931,944,962,918,874,925
2,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,6499,7394,903,917,931,944,962,918,895,924
3,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,6511,7418,904,915,932,944,961,930,907,925
4,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,6538,7475,904,915,935,941,961,957,937,925
5,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,6521,7500,903,916,931,944,962,940,979,925


In [63]:
# map all the columns to seconds
for col in split_cols:
    df[col + "_s"] = df[col].map(to_seconds)

This is the stuff I did after you two left on 11/7.  Basically: we need a function that can subtract the previous rows.

In [78]:
# make a pairing of things to subtract
s = [split for split in split_cols if split != "Half"]
split_col_pairs = [(s[i], s[i+1]) for i in range(len(s)-1)]

# check it
split_col_pairs

[('five', 'ten'),
 ('ten', 'fifteen'),
 ('fifteen', 'twenty'),
 ('twenty', 'twentyfive'),
 ('twentyfive', 'thirty'),
 ('thirty', 'thirtyfive'),
 ('thirtyfive', 'forty')]

In [80]:
# check that it works for the first dude

for col1, col2 in split_col_pairs:
    print(df.loc[1,][col2+"_s"] - df.loc[1,][col1+"_s"])

903
916
931
944
962
918
874


In [84]:
# Now do it for everyone
for col1, col2 in split_col_pairs:
    df[col2 + "_split"] = df[col2 + "_s"] - df[col1 + "_s"]
    
#check it
df.head()

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,thirty_s,thirtyfive_s,forty_s,ten_split,fifteen_split,twenty_split,twentyfive_split,thirty_split,thirtyfive_split,forty_split
Overall,,,,,,,,,,,,,,,,,,,,,
1,0,11,"Kirui, Geoffrey",24,M,Keringet,NaN,KEN,NaN,NaN,...,5581,6499,7373,903,916,931,944,962,918,874
2,1,17,"Rupp, Galen",30,M,Portland,OR,USA,NaN,NaN,...,5581,6499,7394,903,917,931,944,962,918,895
3,2,23,"Osako, Suguru",25,M,Machida-City,NaN,JPN,NaN,NaN,...,5581,6511,7418,904,915,932,944,961,930,907
4,3,21,"Biwott, Shadrack",32,M,Mammoth Lakes,CA,USA,NaN,NaN,...,5581,6538,7475,904,915,935,941,961,957,937
5,4,9,"Chebet, Wilson",31,M,Marakwet,NaN,KEN,NaN,NaN,...,5581,6521,7500,903,916,931,944,962,940,979


In [85]:
# check that there's nothing strange going on 
df["thirtyfive_split"].describe()

count    26259.000000
mean      1871.579420
std        416.732748
min        828.000000
25%       1580.000000
50%       1804.000000
75%       2089.500000
max      10720.000000
Name: thirtyfive_split, dtype: float64

Only thing that's missing is you don't have a `five_split` column.  It's the same as `five_s`, but just to make iteration in your script a little easier, let's add that.

In [87]:
df["five_split"] = df["five_s"]

Looks great, let's ship it out.

In [88]:
df.to_csv("marathon_2017_with_seconds.csv")

Enjoy!